In [29]:
# Node2Vec Model Using torch geometric with Cora
import torch
from torch_geometric.datasets import Planetoid
import os
from torch_geometric.nn import Node2Vec
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

torch.cuda.is_available()

True

In [11]:
# DOWNLOAD DATA
os.environ['http_proxy'] = "http://proxy-sifi.rd.corpintra.net:3128" 
os.environ['https_proxy'] = "http://proxy-sifi.rd.corpintra.net:3128"
path = os.getcwd()
dataset = Planetoid(path, "Cora")


Processing...
Done!


In [14]:
# IMPORT DATA
data = dataset[0]
print('Coda: ', data)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

Coda:  Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


In [30]:
# CONSTRUCT THE MODEL
Node2Vec_model = Node2Vec(data.edge_index, embedding_dim=128, walk_length=20, context_size=10,
                 walks_per_node=10, p=1, q=1, num_negative_samples=1,
                 num_nodes=None, sparse=True).to(device)

loader = Node2Vec_model.loader(batch_size=128, shuffle=True, num_workers=4) # for batch training
optimizer = torch.optim.SparseAdam(list(Node2Vec_model.parameters()), # list of parameters
                                    lr = 0.01 # learning rate
                                    )

ImportError: `Node2Vec` requires `torch-cluster`.

In [22]:
# TRAIN FUNCTION
def train():
    Node2Vec_model.train() # set training as true for the model
    total_loss = 0
    for pos_rw, neg_rw in loader:
        optimizer.zero_grad() # reset of gradient of all variables
        loss = Node2Vec_model.loss(pos_rw, neg_rw)
        loss.backward()
        optimizer.step()
        total_loss =+ loss.item()
    return total_loss / len(loader)
    
# OBTAIN EMBEDDING REPRESENTATION
for epoch in range(1,101):
    loss = train()
    print(f'Epoch: {epoch:02d}, Loss: {loss:.4f}')

NameError: name 'Node2Vec_model' is not defined

In [21]:
# PLOT 2D OF EMBEDDED REPRESENTATION
@torch.no_grad() # deactivate autograd functionality
def plot_point(colors):
    Node2Vec_model.eval() # evalutae the model based on the trained parameters
    z = Node2Vec_model(torch.arange(data.num_nodes, device=device)) # embedding rep
    z = TSNE(n_components=2).fit_transform(z.cpu().numpy())
    y = data.y.cpu().numpy()
    plt.figure()
    for i in range(dataset.num_classes):
        plt.scatter(z[y==i,0], z[y==i,1],s=20,color=colors[i])
    plt.axis('off')
    plt.show()
colors = [
        '#ffc0cb', '#bada55', '#008080', '#420420', '#7fe5f0', '#065535',
        '#ffd700'
    ]
plot_point(colors)



NameError: name 'Node2Vec_model' is not defined

In [ ]:
# NODE CLASSIFICATION